In [1]:
import json
import pandas as pd

In [2]:
version = 11

In [3]:
with open("codebook.json", "r") as codebook_file:
    codebook = json.load(codebook_file)

with open("country.json", "r") as country_file:
    countries = json.load(country_file)

df = pd.read_csv(f'ESS{version}/ESS{version}.csv')

In [4]:
df["id"] = df["cntry"] + '_' + df["idno"].astype(str)
df.set_index("id", inplace=True)
df = df[codebook.keys()]

In [5]:
df

,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,...,hmsacld,euftf,lrnobed,loylead,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt
id,,,,,,,,,,,,,,,,,,,,,
AT_50014,1,4,5,4,5,6,9,10,5,5,...,3,6,4,4,2,2,3,7,3,5
AT_50030,2,3,2,3,2,6,6,4,1,0,...,1,9,2,5,2,2,2,6,5,9
AT_50057,2,4,4,4,3,7,5,8,4,4,...,1,7,5,3,1,1,1,9,9,8
AT_50106,3,2,2,2,3,5,6,9,3,3,...,3,5,2,9,2,2,2,6,6,5
AT_50145,2,3,1,4,3,6,8,8,5,5,...,2,10,1,2,1,1,2,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK_86379,1,4,2,4,1,4,5,10,3,3,...,4,10,5,5,1,1,1,10,10,10
SK_86407,2,3,3,3,3,4,7,10,4,6,...,4,7,3,2,2,3,3,8,3,5
SK_86408,2,4,3,3,2,5,4,5,5,5,...,1,7,3,3,2,2,2,7,10,9


In [6]:
def normalize_column(column, values, scale_type):
        
    if scale_type == 'u':
        column = column.apply(lambda x: (x - min(values)) / (max(values) - min(values)) if x not in [77, 88, 99] else x)
    elif scale_type == 'bi':
        column = column.apply(lambda x: 2 * (x - min(values)) / (max(values) - min(values)) - 1 if x not in [77, 88, 99] else x)
    
    return column

In [7]:
for variable in df.columns:
    values = codebook[variable]['values']
    scale_type = codebook[variable]['scale_type']
    
    
    if max(values) < 7:
        df[variable] = df[variable].replace({7: 77, 8: 88, 9: 99})

        df[variable] = normalize_column(df[variable], values, scale_type)   

In [8]:
df

,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,...,hmsacld,euftf,lrnobed,loylead,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt
id,,,,,,,,,,,,,,,,,,,,,
AT_50014,0.000000,0.75,1.00,0.75,1.00,6,9,10,5,5,...,0.0,6,0.5,0.5,-0.333333,-0.333333,0.333333,7,3,5
AT_50030,0.333333,0.50,0.25,0.50,0.25,6,6,4,1,0,...,-1.0,9,-0.5,1.0,-0.333333,-0.333333,-0.333333,6,5,9
AT_50057,0.333333,0.75,0.75,0.75,0.50,7,5,8,4,4,...,-1.0,7,1.0,0.0,-1.000000,-1.000000,-1.000000,9,9,8
AT_50106,0.666667,0.25,0.25,0.25,0.50,5,6,9,3,3,...,0.0,5,-0.5,99.0,-0.333333,-0.333333,-0.333333,6,6,5
AT_50145,0.333333,0.50,0.00,0.75,0.50,6,8,8,5,5,...,-0.5,10,-1.0,-0.5,-1.000000,-1.000000,-0.333333,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK_86379,0.000000,0.75,0.25,0.75,0.00,4,5,10,3,3,...,0.5,10,1.0,1.0,-1.000000,-1.000000,-1.000000,10,10,10
SK_86407,0.333333,0.50,0.50,0.50,0.50,4,7,10,4,6,...,0.5,7,0.0,-0.5,-0.333333,0.333333,0.333333,8,3,5
SK_86408,0.333333,0.75,0.50,0.50,0.25,5,4,5,5,5,...,-1.0,7,0.0,0.0,-0.333333,-0.333333,-0.333333,7,10,9


In [9]:
df.to_csv("preprocessed_data.csv")